# Домашнее задание 1

Это домашнее задание по материалам 1-3 недели семестра. Дедлайн по отправке - __23:59 17 марта__.

- Домашнее задание выполняется в этом же Jupyter Notebook'e.

- Файл необходимо переименовать: __Фамилия_Имя__ (без пробелов в начале и конце). Пример: __Иванов_Иван__.

- ДЗ нужно отправлять на __OptimizationHomework@yandex.ru__. Тема письма: __ИАД_номер задания__ (без пробелов в начале и конце). Для данного ДЗ тема письма: __ИАД_1__.

- Для решения можно использовать Google Colab, но присылать нужно не ссылку на Colab, а готовый notebook и все необходимые дополнительные файлы.

- Решение каждой задачи/пункта задачи поместите после условия.

- Не забывайте добавлять необходимые пояснения и комментарии.

- В финальной версии, которая будет отправлена на проверку, должны быть удалены все отладочные артефакты. Под таким артефактами подразумеваются любые выводы ячеек, которые никак не прокоментированы в тексте, а также любой массовый/длинный технический вывод (даже если он прокомментирован в тексте).

- При полном запуске решения (Kernel -> Restart & Run All) все ячейки должны выполняться без ошибок.

- Максимальный балл за задание - 140. Для получения полного балла за домашнее задание необходимо набрать 100 баллов.

Желаем успехов!

Рассмотрим задачу минимизации эмпирического риска:
\begin{equation}
\min_{w \in \mathbb{R}^d} \frac{1}{n} \sum\limits_{i=1}^n \ell (g(w, x_i), y_i) + \frac{\lambda}{2} \| w \|^2_2,
\end{equation}
где $\ell$ - функция потерь, $g$ - модель, $w$ - параметры модели, $\{x_i, y_i\}_{i=1}^n$ - выборка данных из векторов признаков $x_i$ и меток $y_i$, $\lambda > 0$ - параметр регуляризации.

Используем линейную модель $g(w, x) = w^T x$ и логистическую/сигмоидную функцию потерь: $\ell(z,y) = \ln (1 + \exp(-yz))$ (__Важно: $y$ должен принимать значения $-1$ или $1$__). Полученная задача называется задачей логистической регрессии. 

__Задача 1. (всего 20 баллов)__ Проведем подготовительную работу. 

__а). (15 баллов)__ Покажите, что градиент и гессиан целевой функции можно записать в виде:
$$
\nabla f(w) = \frac{1}{n} \sum_{i=1}^n \frac{-y_i}{1 + \exp(y_i w^Tx_i)}x_i + \lambda w,
\quad
\nabla^2 f(w) = \frac{1}{n} \sum_{i=1}^n \frac{\exp(y_i w^Tx_i)}{(1 + \exp(y_i w^Tx_i))^2} x_i x_i^T + \lambda I.
$$

Докажите, что решаемая задача является $\mu$-сильно выпуклой и имеет $L$-Липшицев градиент с $\mu = \lambda$ и $L = \lambda + \frac{1}{4n} \sum_{i=1}^n \| x_i\|^2_2$. 

In [1]:
#ваше решение (Markdown)

К заданию приложен датасет _mushrooms_. С помощью следующего кода сформируйте матрицу $X$ и вектор $y$, в которой и будет храниться выборка $\{x_i, y_i\}_{i=1}^n$:

In [1]:
dataset = "mushrooms.txt" 
#файл должен лежать в той же деректории, что и notebook

In [2]:
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]

Поменяем вектор $y$, чтобы $y_i$ принимали значения $-1$ и $1$. Вы также можете сделать дополнительную предобработку данных (приемами из машинного обучения), но это никак дополнительно не оценивается.

In [4]:
y = 2 * y - 3

Разделим данные на две части: обучающую и тестовую.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

__б). (5 балла)__ Для обучающей части $X_{train}$, $y_{train}$ оцените константу $L$. Задайте $\lambda$ так, чтобы $\lambda \approx L / 1000$.  Реализуйте в коде подсчет значения, градиента и гессиана для нашей целевой функции ($X$, $y$, $\lambda$ лучше подавать в качестве параметра, чтобы была возможность их менять, а не только подставлять фиксированные $X_{train}$, $y_{train}$). Можно использовать как библиотеку ``numpy``, так и более специализированными библиотеки, с которыми знакомы, например, ``autograd``, ``pytorch``, ``jax``.

In [ ]:
#ваше решение (Code и Markdown)

__Задача 2. (всего 30 баллов)__ Данная часть задания связана с моментумом и ускорением.

__а). (10 баллов)__ Реализуйте метод тяжелого шарика и ускоренный градиентный метод Нестерова. 

На всякий случай мы приводим здесь вариант описания функции для некоторого метода. Можно пользоваться таким форматом по желанию. Учтите, что в коде встречается ``x_sol`` - это проблему стоит как-то обойти или не использовать критерии, завязанные на ``x_sol``.

In [ ]:
def method(f, nabla_f, x_0, x_sol, gamma_k,
                     K = 10**3, eps = 10**-5, mode = 'x_k - x^*'):
    '''
        f - целевая функция
        nabla_f - градиент целевой функции
        x_0 - стартовая точка
        x_sol - точное решение (оно нужно для подсчета ошибки)
        gamma_k - функция для вычисления шага метода
        K - количество итераций (по умолчанию 1е3)
        eps - желаемая точность (по умолчанию 1е-5)
        mode - критерий сходимости 
               Значения либо 'x_k - x^*' - тогда критерий сходимости будет ||x_k - x^*||,
               либо 'f(x_k) - f(x^*)' - тогда критерий сходимости будет f(x_k) - f(x^*),
               либо 'x_k+1 - x_k', либо 'f(x_k+1) - f(x_k)' (критерии будут аналогичными)

        Функция возвращает точку, в которой достигается минимум и вектор ошибок
    '''
    return

In [ ]:
#ваше решение (Code и Markdown)

__б). (20 баллов)__ Решите задачу оптимизации на тестовой выборке с помощью двух реализованных методов. Зафиксируйте шаг $\frac{1}{L}$ и перебирайте разные значения моментума от -1 до 1 (5 хватит). Проверьте также значения моментума равные $\frac{k}{k+3}$, $\frac{k}{k+2}$, $\frac{k}{k+1}$ ($k$ - номер итерации), а если целевая функция является  сильно выпуклой, то и $\frac{\sqrt{L} - \sqrt{\mu}}{\sqrt{L} + \sqrt{\mu}}$. Стартовую точку и критерий сходимости можете выбрать на свой вкус, мы советуем использовать нормированную версию критерия, например, $\frac{\| \nabla f(x^k) \|}{\| \nabla f(x^0) \|}$, а также использовать в Задачах 3-4 ту же самую стартовую точку и тот же самый критерий сходимости.

В данном пункте нужно построить три графика: 1) значения критерия сходимости от номера итерации для метода тяжелого шарика с различными значениями моментума, 2) значения критерия сходимости от номера итерации для ускоренного градиентного метода с различными значениями моментума, 3) значения критерия сходимости от номера итерации для двух методов с наилучшим выбором моментума для каждого, а также градиентного спуска.

Не забывайте делать выводы и комментировать результаты. Например, отразите всегда ли сходимость является монотонной?

In [ ]:
#ваше решение (Code и Markdown)

__Задача 3. (всего 40 + 30 баллов)__ Теперь поговорим про метод Ньютона и квазиньютоновские методы.

__а). (20 баллов)__ Для задачи регресии реализуйте классический метод Ньютона и запустите его. Сходится ли он? Если нет, то попробуйте перед использованием метода Ньютона сначала запускать метод градиентного спуска на несколько итераций. Варьируйте количество шагов градиентного спуска. Постройте график значения критерия сходимости от номера итерации для комбинации градиентного спуска и метода Ньютона с различным числом шагов градиентного спуска. Сделайте вывод. 

In [ ]:
#ваше решение (Code и Markdown)

__б). (20 + 30 баллов)__ Для данной задачи реализуйте квазиньютоновский метод BFGS (можно реализовать более продвинутую версию L-BFGS, посмотрев оригинальную [статью](http://users.iems.northwestern.edu/~nocedal/PDFfiles/limited-memory.pdf) или лучше параграф 9.1 [книги](https://www.ime.unicamp.br/~pulino/MT404/TextosOnline/NocedalJ.pdf)). За реализацию L-BFGS и объяснение, как это сделать правильно и вычислительно эффективно можно получить еще __30 баллов__. Используйте метод(ы) для решения задачи регресии. Добавьте его(их) на график из предыдущего пункта. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)

__Задача 4. (20 баллов)__ Осталось объеденить результаты полученные в Задачах 1-3. Для этого вспомним, что исходная задача регрессии является задачой машинного обучения и с помощью линейной модели $g$ можно предсказывать значения меток $y$. Как использовать итоговую модель для предсказания? Ответив на вопрос, сделайте предсказания на тестовой выборке $X_{test}$. Сравните с реальными метками $y_{test}$. Количество правильно угаданных меток есть точность/accuracy модели. Сравните метод градиентного спуска, метод тяжелого шарика, ускоренный градиентный метод, метод Ньютона, BFGS(L-BFGS). Постройте два графика: значение критерия сходимости от времени работы и точность предсказания от времени работы. Сделайте вывод.

In [ ]:
#ваше решение (Code и Markdown)